<a href="https://colab.research.google.com/github/JiwooLee0822/transformer/blob/main/Encoder%26Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

## Encoder Layer

multi-head self attention layer와 position-wise fully connected feed-forward network layer가 사용됨.

각 layer의 output의 demension은 embedding layer와 동일한 512이다.

https://code-angie.tistory.com/10

In [3]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, head, d_ff, dropout):
    super().__init__()
    self.attention = MultiHeadAttention(d_model, head)
    self.layerNorm1 = nn.LayerNorm(d_model)

    self.ffn = PositionWiseFeedForward(d_model, d_ff)
    self.layerNorm2 = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(p=dropout)

  def forward(self, x, padding_mask):

    # multi-head attention (self attention)
    attention = self.attention(q = x, k = x, v = x, mask = padding_mask)
    # add & norm
    attention = self.layerNorm1(attention + x)
    attention = self.dropout(attention)
    # feed-forward network
    outputs = self.ffn(attention)
    # add&norm
    outputs = self.layerNorm2(outputs + attention)
    outputs = self.dropout(outputs)

    return outputs



## Encoder

1) 입력에 대한 input embedding, posiional encoding 생성

2) add & dropout - input embedding과 positional encoding 더한 후, dropout 수행

3) n번 EncoderLayer 반복하기 (n_layer)

In [4]:
class Encoder(nn.Module):
  def __init__(self, n_input_vocab, d_model, head, d_ff, max_len, padd_idx, dropout, n_layers, device):
    super().__init__()

    # embedding
    self.input_emb = nn.Embedding(n_input_vocab, d_model, padding_idx=padd_idx)
    self.pos_encoding = PositionalEncoding(d_model, max_len, device)
    self.dropout = nn.Dropout(p=dropout)

    # n개의 encoder layer를 list에 담기
    self.encoder_layers = nn.ModuleList([EncoderLayer(d_model=d_model,
                                                      head=head,
                                                      d_ff=d_ff,
                                                      dropout=dropout)
                                          for _ in range(n_layers)])

    def forward(self, x, padding_mask):
      # 입력에 대한 input embedding, positional encoding 생성
      input_emb = self.input_emb(x)
      pos_encoding = self.pos_encoding(x)

      # add & dropout
      x = self.dropout(input_emb + pos_encoding)

      # n번 encoderlayer 반복하기
      for encoder_layer in self.encoder_layers:
        outputs = encoder_layer(pos_inputs, src_mask)

      return outputs


